In [19]:
import cv2
import pandas as pd
import os
import numpy as np
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def segmentarImg(input_folder):
    x_values = []

    # Leer imágenes de la carpeta de entrada
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            file_path = os.path.join(input_folder, filename)
            img = cv2.imread(file_path)
            img_gray = cv2.imread(file_path, 0)

            # Aplicar Otsu para binarización
            _, binary_image = cv2.threshold(img_gray, 0, 255, cv2.THRESH_OTSU)

            # Etiquetado de blobs
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

            print(f'Numero de objetos encontrados en {filename}: {num_labels-1}')

            # Procesa cada blob detectado
            for label in range(1, num_labels):
                x, y, w, h, _ = stats[label]
                img_segment = binary_image[y:y+h, x:x+w]

                img_segment = resize(img_segment, (100, 100), anti_aliasing=True)
                x_values.append(img_segment)

    x_values = np.array(x_values).astype('float32')
    x_values = np.expand_dims(x_values, axis=-1)
    return x_values

def CNN_classifier():
    model = Sequential([
      Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
      MaxPooling2D((2, 2)),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Flatten(),
      Dense(64, activation='relu'),
      Dropout(0.5),
      Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

y_train = np.array([1,1,0,2,2,2,2,0,1,1,1,1,0,2,2,1,1,0,1,1,2,2,0,2,2,
                    0,1,0,2,1,0,2,0,1,2,0,1,0,2,1,2,2,0,1,0,2,1,0,2,0,
                    1,2,1,2,1,0,0,0,0,0,2,1,2,1,2,0,0,0,0,0,1,2,1,2,1,
                    1,2,1,0,1,0,1,2,0,0,0,0,0,1,1,2,1,2,0,2,2,0,1,2,2])
X_train = segmentarImg('train')
model = CNN_classifier()

model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

Numero de objetos encontrados en train1.jpg: 25
Numero de objetos encontrados en train3.jpg: 25
Numero de objetos encontrados en train4.jpg: 25
Numero de objetos encontrados en train2.jpg: 25
Epoch 1/100
8/8 [==============================] - 2s 162ms/step - loss: 1.0928 - accuracy: 0.3875 - val_loss: 1.0902 - val_accuracy: 0.4000
Epoch 2/100
8/8 [==============================] - 1s 136ms/step - loss: 1.0305 - accuracy: 0.5375 - val_loss: 1.1340 - val_accuracy: 0.3500
Epoch 3/100
8/8 [==============================] - 1s 135ms/step - loss: 1.0187 - accuracy: 0.5250 - val_loss: 1.1115 - val_accuracy: 0.3500
Epoch 4/100
8/8 [==============================] - 1s 157ms/step - loss: 0.9950 - accuracy: 0.5750 - val_loss: 1.1546 - val_accuracy: 0.3500
Epoch 5/100
8/8 [==============================] - 2s 220ms/step - loss: 0.9894 - accuracy: 0.5625 - val_loss: 1.1133 - val_accuracy: 0.3500
Epoch 6/100
8/8 [==============================] - 2s 201ms/step - loss: 0.9827 - accuracy: 0.5250 - va

In [21]:
from joblib import dump
dump(model, '[modelo]CNN_clasificador.joblib')

['[modelo]CNN_clasificador.joblib']

In [20]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_train, y_train)
print(f'Pérdida: {loss}, Precisión: {accuracy}')

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


4/4 [==============================] - 0s 74ms/step - loss: 1.0639 - accuracy: 0.7300
Pérdida: 1.0639022588729858, Precisión: 0.7300000190734863


In [28]:
import cv2
import numpy as np
import pandas as pd
import os
from joblib import load

def show_labeled_images(input_folder, output_folder, classifier):
    # Limpiar la carpeta de salida
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    else:
        for file in os.listdir(output_folder):
            os.remove(os.path.join(output_folder, file))

    # Procesar cada archivo en la carpeta de entrada
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            file_path = os.path.join(input_folder, filename)
            img = cv2.imread(file_path)
            img_gray = cv2.imread(file_path, 0)

            # Aplicar Otsu para binarización
            _, binary_image = cv2.threshold(img_gray, 0, 255, cv2.THRESH_OTSU)

            # Etiquetado de componentes conectados
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

            predictions = []

            for label in range(1, num_labels):
                x, y, w, h, _ = stats[label]
                segment = binary_image[y:y+h, x:x+w]

                segment = resize(segment, (100, 100), anti_aliasing=True)
                segment = np.array(segment).astype('float32')
                segment = np.expand_dims(segment, axis=0)
                segment = np.expand_dims(segment, axis=-1)
                prediction = classifier.predict(segment)
                prediction = np.argmax(prediction)
                predictions.append(prediction)

                # Dibujar rectángulo y etiqueta en la imagen
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(img, str(prediction), (x + 20, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Guardar la imagen procesada en la carpeta de salida
            output_image_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_image_path, img)
            print(f"Imagen {filename} guardada...")

classifier = load('[modelo]CNN_clasificador.joblib')
# Deteccion de fichas en las imagenes
show_labeled_images("dataset", "CNNlabelling", classifier)

1/1 [==============================] - 0s 22ms/step
Imagen test26.jpg guardada...
1/1 [==============================] - 0s 38ms/step
Imagen test16.jpg guardada...
1/1 [==============================] - 0s 28ms/step
Imagen test46.jpg guardada...
1/1 [==============================] - 0s 29ms/step
Imagen test92.jpg guardada...
1/1 [==============================] - 0s 22ms/step
Imagen test98.jpg guardada...
1/1 [==============================] - 0s 22ms/step
Imagen test91.jpg guardada...
1/1 [==============================] - 0s 22ms/step
Imagen test93.jpg guardada...
1/1 [==============================] - 0s 37ms/step
Imagen test63.jpg guardada...
1/1 [==============================] - 0s 22ms/step
Imagen test78.jpg guardada...
1/1 [==============================] - 0s 22ms/step
Imagen test36.jpg guardada...
1/1 [==============================] - 0s 23ms/step
Imagen test76.jpg guardada...
1/1 [==============================] - 0s 22ms/step
Imagen test13.jpg guardada...
1/1 [===========